# Campeonato Brasileiro - Série A

## Radom Forest Classifier

### 1. Import packages

In [119]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [120]:
from sklearn.ensemble import RandomForestClassifier

In [121]:
from sklearn.ensemble import AdaBoostClassifier

In [122]:
from sklearn.model_selection import cross_val_score

In [123]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score, recall_score
from sklearn.metrics import f1_score

In [124]:
from sklearn import datasets

### 2. Reading data

In [125]:
jogos = pd.read_csv("data/jogos-resultado-posicao.csv",sep=",")

In [126]:
del jogos["Unnamed: 0"]

In [127]:
#jogos.query('ano > 2012', inplace=True)

In [128]:
jogos.head(3)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado,posicao_m,posicao_v,aproveitamento_5r_v,aproveitamento_5r_m,aproveitamento_10r_v,aproveitamento_10r_m
0,brasileiro-a,2017,30,23-10,Botafogo,Corinthians,2,1,MANDANTE,6.0,1.0,0.40,0.47,0.40,0.63
1,brasileiro-a,2017,30,22-10,Chapecoense,Fluminense,2,0,MANDANTE,11.0,13.0,0.47,0.47,0.40,0.43
2,brasileiro-a,2017,30,22-10,Ponte Preta,Avaí,1,2,VISITANTE,18.0,16.0,0.13,0.27,0.43,0.30


In [129]:
jogos.keys()

Index(['campeonato', 'ano', 'rodada', 'data', 'mandante', 'visitante',
       'gols_mandante', 'gols_visitante', 'resultado', 'posicao_m',
       'posicao_v', 'aproveitamento_5r_v', 'aproveitamento_5r_m',
       'aproveitamento_10r_v', 'aproveitamento_10r_m'],
      dtype='object')

In [130]:
times = np.concatenate((jogos["visitante"].unique(), jogos["mandante"].unique()), axis=0)

In [131]:
times_dict = dict(zip(times, range(1, len(times))))

In [219]:
times_dict

{'América-MG': 65,
 'América-RN': 78,
 'Atlético-GO': 60,
 'Atlético-MG': 59,
 'Atlético-PR': 45,
 'Avaí': 56,
 'Bahia': 46,
 'Barueri': 72,
 'Botafogo': 42,
 'Brasiliense': 40,
 'Ceará': 71,
 'Chapecoense': 43,
 'Corinthians': 57,
 'Coritiba': 61,
 'Criciúma': 68,
 'Cruzeiro': 47,
 'Figueirense': 62,
 'Flamengo': 52,
 'Fluminense': 58,
 'Fortaleza': 79,
 'Goias': 66,
 'Grêmio': 48,
 'Guarani': 73,
 'Internacional': 64,
 'Ipatinga': 75,
 'Joinville': 67,
 'Juventude': 76,
 'Náutico': 70,
 'Palmeiras': 54,
 'Paraná': 77,
 'Paysandu': 81,
 'Ponte Preta': 44,
 'Portuguesa': 69,
 'Santa Cruz': 63,
 'Santo Andre': 74,
 'Santos': 49,
 'Sao Caetano': 80,
 'Sport Recife': 53,
 'São Paulo': 50,
 'Vasco': 51,
 'Vitória': 55}

In [132]:
jogos = jogos.applymap(lambda s: times_dict.get(s) if s in times_dict else s)

### 3. Define target and data subset

In [133]:
jogos_X_df = jogos.copy()

In [134]:
del jogos_X_df["resultado"]

In [135]:
del jogos_X_df["campeonato"]

In [136]:
del jogos_X_df["data"]

In [137]:
del jogos_X_df["gols_mandante"]

In [138]:
del jogos_X_df["gols_visitante"]

In [139]:
del jogos_X_df["aproveitamento_5r_v"]

In [140]:
del jogos_X_df["aproveitamento_5r_m"]

In [141]:
jogos_X_df.head()

,ano,rodada,mandante,visitante,posicao_m,posicao_v,aproveitamento_10r_v,aproveitamento_10r_m
0,2017,30,42,57,6.0,1.0,0.40,0.63
1,2017,30,43,58,11.0,13.0,0.40,0.43
2,2017,30,44,56,18.0,16.0,0.43,0.30
3,2017,30,45,53,9.0,15.0,0.23,0.40
4,2017,30,46,55,12.0,17.0,0.47,0.40


In [142]:
jogos_X_df.keys()

Index(['ano', 'rodada', 'mandante', 'visitante', 'posicao_m', 'posicao_v',
       'aproveitamento_10r_v', 'aproveitamento_10r_m'],
      dtype='object')

In [143]:
y = jogos["resultado"].values

In [144]:
X = jogos_X_df.values

### 4. Train and Test Split

In [145]:
from sklearn.model_selection import train_test_split

In [146]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y, 
                                        test_size=0.2, 
                                        random_state=1)

### 4.1 Grid Search

In [29]:
from sklearn.grid_search import GridSearchCV

/home/thiago/workspace/python/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/thiago/workspace/python/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [30]:
from sklearn.datasets import make_classification

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True)

In [33]:
param_grid = {     
    'n_estimators': [800],
    'min_samples_leaf': [10],
    'max_features': ['log2'],
    'n_jobs': [-1],
    'min_samples_split': [3],
    'warm_start': [False],
    'oob_score':  [False], 
    'bootstrap': [True],
    'criterion': ['entropy'],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

In [34]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)

In [35]:
CV_rfc.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [800], 'min_samples_leaf': [10], 'max_features': ['log2'], 'n_jobs': [-1], 'min_samples_split': [3], 'warm_start': [False], 'oob_score': [False], 'bootstrap': [True], 'criterion': ['entropy'], 'class_weight': ['balanced', 'balanced_subsample', None]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [36]:
print (CV_rfc.best_params_)

{'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 10, 'min_samples_split': 3, 'n_estimators': 800, 'n_jobs': -1, 'oob_score': False, 'warm_start': False}


### 5. Apply Random Forest Classifier

In [147]:
rf_model = RandomForestClassifier(random_state=1, criterion='entropy', max_features='log2', n_estimators=800, min_samples_leaf=10)

In [148]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

### 6. Model Cross-Validation

In [149]:
cv_scores_rf = cross_val_score(rf_model, X_train, y_train, 
                               cv=5,scoring="accuracy")
cv_scores_rf

array([ 0.60227273,  0.58677686,  0.59710744,  0.57394002,  0.59274611])

In [150]:
cv_scores_rf_mean =  np.mean(cv_scores_rf)
cv_scores_rf_mean

0.59056863189310982

### Apply AdaBoostClassifier

In [41]:
abc_model = AdaBoostClassifier(random_state=1, n_estimators=50, learning_rate=1., algorithm="SAMME.R")

In [42]:
abc_model.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=1)

In [43]:
cv_scores_abc = cross_val_score(abc_model, X_train, y_train, 
                               cv=5,scoring="accuracy")
cv_scores_abc

array([ 0.60020661,  0.58264463,  0.56404959,  0.56670114,  0.59689119])

In [44]:
cv_scores_abc_mean =  np.mean(cv_scores_abc)
cv_scores_abc_mean

0.58209863113898097

### 7. Model Test

In [151]:
y_test_pred = rf_model.predict(X_test)

In [152]:
y_test.shape

(1210,)

In [153]:
# Results
pd.crosstab(y_test, y_test_pred,
            rownames=['Actual'], 
            colnames=['Predicted'])

Predicted,EMPATE,MANDANTE,VISITANTE
Actual,,,
EMPATE,27,210,75
MANDANTE,17,547,53
VISITANTE,17,132,132


In [154]:
confusion_matrix(y_test, y_test_pred)

array([[ 27, 210,  75],
       [ 17, 547,  53],
       [ 17, 132, 132]])

In [155]:
accuracy_score_test = accuracy_score(y_test, y_test_pred)
accuracy_score_test

0.58347107438016532

In [156]:
#precision_score_forest = precision_score(y_test, y_test_pred)
#precision_score_forest

In [157]:
#recall_score_forest = recall_score(y_test, y_test_pred)
#recall_score_forest

In [158]:
#f1_score_forest = f1_score(y_test, y_test_pred)
#f1_score_forest

### 8. Comparison accuracy: Validation vs Test

In [159]:
comparison = {"Validation": [cv_scores_rf_mean],
             "Test": [accuracy_score_test]}
pd.DataFrame(comparison, index = ["Accuracy"])

,Test,Validation
Accuracy,0.583471,0.590569


### 9. Feature Importance

In [160]:
rf_model.feature_importances_

array([ 0.06709756,  0.13378069,  0.08373038,  0.0852613 ,  0.20689107,
        0.20897494,  0.09987478,  0.11438927])

In [161]:
# View a list of the features and their importance scores
imp_list = list(zip(jogos_X_df.keys(), 
                    rf_model.feature_importances_))
imp_df = pd.DataFrame(imp_list, columns = ["Features", "Importance"])
imp_df.sort_values(by = "Importance", ascending = False)

,Features,Importance
5,posicao_v,0.208975
4,posicao_m,0.206891
1,rodada,0.133781
7,aproveitamento_10r_m,0.114389
6,aproveitamento_10r_v,0.099875
3,visitante,0.085261
2,mandante,0.083730
0,ano,0.067098


### 10. Predict

In [223]:
rodada = pd.read_csv("data/rodada.csv",sep=",")

In [224]:
rodada

,mandante,visitante,placar
0,Palmeiras,Cruzeiro,NaN
1,Avaí,Grêmio,NaN
2,Sport Recife,Coritiba,NaN
3,Vitória,Atlético-GO,NaN
4,Atlético-MG,Botafogo,NaN
5,Fluminense,Bahia,NaN
6,Ponte Preta,Corinthians,NaN
7,Atlético-PR,Chapecoense,NaN
8,Flamengo,Vasco,NaN
9,São Paulo,Santos,NaN


In [225]:
rodada_DF = rodada.copy()

In [226]:
rodada_DF = rodada_DF.applymap(lambda s: times_dict.get(s) if s in times_dict else s)

In [227]:
rodada_DF

,mandante,visitante,placar
0,54,47,NaN
1,56,48,NaN
2,53,61,NaN
3,55,60,NaN
4,59,42,NaN
5,58,46,NaN
6,44,57,NaN
7,45,43,NaN
8,52,51,NaN
9,50,49,NaN


In [228]:
jogos.head(3)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado,posicao_m,posicao_v,aproveitamento_5r_v,aproveitamento_5r_m,aproveitamento_10r_v,aproveitamento_10r_m
0,brasileiro-a,2017,30,23-10,42,57,2,1,MANDANTE,6.0,1.0,0.40,0.47,0.40,0.63
1,brasileiro-a,2017,30,22-10,43,58,2,0,MANDANTE,11.0,13.0,0.47,0.47,0.40,0.43
2,brasileiro-a,2017,30,22-10,44,56,1,2,VISITANTE,18.0,16.0,0.13,0.27,0.43,0.30


In [229]:
jogos.query("visitante == 54 and rodada == 30 and ano == 2017")

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado,posicao_m,posicao_v,aproveitamento_5r_v,aproveitamento_5r_m,aproveitamento_10r_v,aproveitamento_10r_m
6,brasileiro-a,2017,30,22-10,48,54,1,3,VISITANTE,4.0,2.0,0.67,0.47,0.6,0.37


In [230]:
ano = jogos["ano"].max()

In [231]:
rod = jogos[jogos["ano"] == ano]["rodada"].max()

In [232]:
rodada_DF["ano"] = ano

In [233]:
rodada_DF["rodada"] = rod + 1

In [234]:
def getPosicao(jogos, ano, rodada, time):    

    ultimo_jogo = jogos.query("ano == " + str(ano) + " and rodada == " + str(rodada) + " and mandante == " + str(time))
    if not ultimo_jogo.empty:
        return ultimo_jogo["posicao_m"].values[0]
    
    ultimo_jogo = jogos.query("ano == " + str(ano) + " and rodada == " + str(rodada) + " and visitante == " + str(time))        
    return ultimo_jogo["posicao_v"].values[0]

In [235]:
def getAproveitamento(jogos, ano, rodada, time):    

    ultimo_jogo = jogos.query("ano == " + str(ano) + " and rodada == " + str(rodada) + " and mandante == " + str(time))
    if not ultimo_jogo.empty:
        return ultimo_jogo["aproveitamento_10r_m"].values[0]
    
    ultimo_jogo = jogos.query("ano == " + str(ano) + " and rodada == " + str(rodada) + " and visitante == " + str(time))        
    return ultimo_jogo["aproveitamento_10r_v"].values[0]

In [236]:
rodada_DF["posicao_m"] = rodada_DF.apply(lambda row: getPosicao(jogos, ano, rod, row["mandante"]),axis=1)

In [237]:
rodada_DF["posicao_v"] = rodada_DF.apply(lambda row: getPosicao(jogos, ano, rod, row["visitante"]),axis=1)

In [238]:
rodada_DF["aproveitamento_10r_m"] = rodada_DF.apply(lambda row: getAproveitamento(jogos, ano, rod, row["mandante"]),axis=1)

In [239]:
rodada_DF["aproveitamento_10r_v"] = rodada_DF.apply(lambda row: getAproveitamento(jogos, ano, rod, row["visitante"]),axis=1)

In [240]:
del rodada_DF["placar"]

In [241]:
rodada_DF

,mandante,visitante,ano,rodada,posicao_m,posicao_v,aproveitamento_10r_m,aproveitamento_10r_v
0,54,47,2017,31,2.0,5.0,0.60,0.67
1,56,48,2017,31,16.0,4.0,0.43,0.37
2,53,61,2017,31,15.0,19.0,0.23,0.20
3,55,60,2017,31,17.0,20.0,0.47,0.47
4,59,42,2017,31,10.0,6.0,0.50,0.63
5,58,46,2017,31,13.0,12.0,0.40,0.40
6,44,57,2017,31,18.0,1.0,0.30,0.40
7,45,43,2017,31,9.0,11.0,0.40,0.43
8,52,51,2017,31,7.0,8.0,0.57,0.60
9,50,49,2017,31,14.0,3.0,0.50,0.50


In [259]:
rodada["prediction_RFC"] = rf_model.predict(rodada_DF[["ano", "rodada", "mandante", "visitante", "posicao_m", "posicao_v", "aproveitamento_10r_m", "aproveitamento_10r_v"]].values)

In [260]:
rodada["prediction_ABC"] = abc_model.predict(rodada_DF[["ano", "rodada", "mandante", "visitante", "posicao_m", "posicao_v", "aproveitamento_10r_m", "aproveitamento_10r_v"]].values)

In [261]:
rodada

,mandante,visitante,placar,resultado,resultado_RFC,resultado_ABC,prediction_RFC,prediction_ABC
0,Palmeiras,Cruzeiro,NaN,MANDANTE,MANDANTE,MANDANTE,MANDANTE,MANDANTE
1,Avaí,Grêmio,NaN,VISITANTE,VISITANTE,VISITANTE,VISITANTE,VISITANTE
2,Sport Recife,Coritiba,NaN,MANDANTE,MANDANTE,MANDANTE,MANDANTE,MANDANTE
3,Vitória,Atlético-GO,NaN,MANDANTE,MANDANTE,MANDANTE,MANDANTE,MANDANTE
4,Atlético-MG,Botafogo,NaN,VISITANTE,MANDANTE,VISITANTE,MANDANTE,VISITANTE
5,Fluminense,Bahia,NaN,MANDANTE,MANDANTE,MANDANTE,MANDANTE,MANDANTE
6,Ponte Preta,Corinthians,NaN,VISITANTE,VISITANTE,VISITANTE,VISITANTE,VISITANTE
7,Atlético-PR,Chapecoense,NaN,MANDANTE,MANDANTE,MANDANTE,MANDANTE,MANDANTE
8,Flamengo,Vasco,NaN,MANDANTE,MANDANTE,MANDANTE,MANDANTE,MANDANTE
9,São Paulo,Santos,NaN,VISITANTE,VISITANTE,VISITANTE,VISITANTE,VISITANTE
